Author: Cezary Januszek

*Created on Saturday, November 11th 2023*
# [Swisscom challenge] Flight delays prediction

#### Research question: *What is the influence of the different airlines/airports on the flight delays?*

The goal of this challenge is to investigate US flight delays and cancellations data from 2015.

First some exploratory data analysis is done to try to find some insights from general statistics about the airlines, airports and average flight delays. Second, using this same data the goal consists in creating a flight delay prediction model by implementing a:
1) baseline model 
2) complex model (Neural Network or other Deep Learning method)

The second model should have better performance than the baseline due to its increased complexity, which will make it also more difficult to explain its predictions.Therefore, an interpretability framework will be added to the second model to allow explicability of the created predictions.

# III. Baseline predicton model

In this notebook, we build the baseline flight delay prediction model. As mentioned in the previous steps, our delays are defined only by the arrival delays of the flights. Moreover, the arrival delays are probably correlated with the departure delays as the already provide important information about the potential final delay of the flight. Therefore, we will check the correlation between these two delay measures and decide to drop the departure delay feature based on the outcome, to ensure that we do not provide to our model to much information about the predicted value (arrival delay).

To solve our flight prediction problem, a **regression model** will be able to not only predict if a flight delay will occur but also to provide a prediction of the time of such a delay. 
The pipeline to build our baseline regression model is as follows:
1. Data pre-processing for feature cleaning and extraction
2. Train-test split
3. Choosing the model and hyperparameters tuning
4.



-----------------------------
Glossary of *flights_df* feature columns:
   - *AIRLINE*: IATA CODE for the airline
   - *ORIGIN_AIRPORT and DESTINATION_AIRPORT*: origin and destination airport of the flight
   - *SCHEDULED_DEPARTURE and SCHEDULED_ARRIVAL*: scheduled times and date of flight departure and landing
   - *DEPARTURE_TIME and ARRIVAL_TIME*: actual real times of flight takeoff and landing
   - *DATE_OF_WEEK*: day of week of scheduled flight
   - *DEPARTURE_DELAY and ARRIVAL_DELAY*: differences between scheduled and real times of flight departure and arrival
   - *DISTANCE*: travel distance between origin and destination airports (in miles)
   - *SCHEDULED_TIME*: scheduled travel time between origin and destination airports
   - *TAIL_NUMBER*: code to identify a specific plane

In [2]:
# Imports
%load_ext autoreload
%autoreload 2

from IPython.core.pylabtools import figsize
figsize(10, 8)

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import datetime
import plotly.express as px
from helper_functions import *

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [11]:
# Load data
flights_df = pd.read_pickle(os.path.abspath('./Data/flights.pkl'))
airport_df = pd.read_pickle(os.path.abspath('./Data/airports.pkl'))
airlines_df = pd.read_csv(os.path.abspath('./Data/airlines.csv'))
                          

In [12]:
airlines_df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


### Data pre-processing for regression

For the task of regression we need to extract some information from dates and hours to put them in simple numerical values as well as we should encode categorical factors (airline, airport).

In [13]:
# extract day, month from dates and hour, minutes from time
flights_df['DEPARTURE_DAY'] = flights_df['DATE'].apply(lambda x: x.day)
flights_df['DEPARTURE_MONTH'] = flights_df['DATE'].apply(lambda x: x.month)

flights_df['ARRIVAL_DAY'] = flights_df['ARRIVAL_DATE'].apply(lambda x: x.day)
flights_df['ARRIVAL_MONTH'] = flights_df['ARRIVAL_DATE'].apply(lambda x: x.month)

flights_df['SCHEDULED_DEPARTURE_HOUR'] = flights_df['SCHEDULED_DEPARTURE'].apply(lambda x: x.hour)
flights_df['SCHEDULED_DEPARTURE_MIN'] = flights_df['SCHEDULED_DEPARTURE'].apply(lambda x: x.minute)

flights_df['SCHEDULED_ARRIVAL_HOUR'] = flights_df['SCHEDULED_ARRIVAL'].apply(lambda x: x.hour)
flights_df['SCHEDULED_ARRIVAL_MIN'] = flights_df['SCHEDULED_ARRIVAL'].apply(lambda x: x.minute)

flights_df = flights_df.drop(['DATE', 'ARRIVAL_DATE', 'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL'], axis=1)

# we also drop DEPARTURE_TIME and DEPARTURE_DELAY as both these features are strongly correlated with our target variable - arrival delay
# ARRIVAL_TIME is dropped as well as it indicates almost directly our arrival delay
flights_df = flights_df.drop(['DEPARTURE_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_TIME'], axis=1)

# TAIL_NUMBER is generally not known to passengers before departure so we drop this column as well
flights_df = flights_df.drop('TAIL_NUMBER', axis=1)